In [9]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np



%matplotlib inline
mpl.rcParams['figure.figsize'] = (16, 10)
pd.set_option('display.max_rows', 500)

import plotly.graph_objects as go

In [10]:
df_test=pd.read_csv('../EDS2/DataForDoublingRate.csv',sep=';')
df_test.head()

,date,location,total_cases
0,2021-03-02,Afghanistan,55759.0
1,2021-03-03,Afghanistan,55770.0
2,2021-03-04,Afghanistan,55775.0
3,2021-03-05,Afghanistan,55827.0
4,2021-03-06,Afghanistan,55840.0


In [11]:
df_test.columns = df_test.columns.to_series().apply(lambda x: x.strip())

In [12]:
df_test.index.name='index'


In [13]:
df_test.head()

,date,location,total_cases
index,,,
0,2021-03-02,Afghanistan,55759.0
1,2021-03-03,Afghanistan,55770.0
2,2021-03-04,Afghanistan,55775.0
3,2021-03-05,Afghanistan,55827.0
4,2021-03-06,Afghanistan,55840.0


In [14]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='total_cases',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='total_cases'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='total_cases'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['location',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['location',filter_on]].groupby(['location']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='total_cases'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['location',filter_on])
    #assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['location']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[[str(filter_on+'_DR')]],left_index=True,right_index=True,how='left')
    #df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('../EDS2/DataForDoublingRate.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'total_cases_filtered')


    mask=pd_result_larg['total_cases']>0
    pd_result_larg['total_cases_filtered_DR']=pd_result_larg['total_cases_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('../EDS2/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['location']=='Germany'].tail())

the test slope is: [2.]
            date location  total_cases  total_cases_filtered  total_cases_DR  \
39969 2022-07-16  Germany   29692989.0            29700518.0      481.965666   
39970 2022-07-17  Germany   29692989.0            29785465.2             inf   
39971 2022-07-18  Germany   29853680.0            29873128.0      370.232965   
39972 2022-07-19  Germany   29994679.0            29990959.8      197.866127   
39973 2022-07-20  Germany   30131303.0            30108791.6      216.071584   

       total_cases_filtered_DR  
39969               380.594116  
39970               363.038414  
39971               345.129140  
39972               290.841553  
39973               254.523480  


In [15]:
print(pd_result_larg[pd_result_larg['location']=='Canada'].tail())

            date location  total_cases  total_cases_filtered  total_cases_DR  \
19223 2022-07-16   Canada    4026044.0             4024124.0     1052.960520   
19224 2022-07-17   Canada    4026044.0             4027813.0     4057.845430   
19225 2022-07-18   Canada    4026069.0             4031192.6   322084.186667   
19226 2022-07-19   Canada    4036848.0             4035255.8      745.955881   
19227 2022-07-20   Canada    4040958.0             4039319.0      541.960508   

       total_cases_filtered_DR  
19223               711.540978  
19224               910.118978  
19225              1139.606108  
19226              1083.307483  
19227               993.122613  


In [16]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())
df_input_large=pd.read_csv('../EDS2/COVID_final_set.csv',sep=';')


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['location'].unique()],
        value=['United States', 'Germany','Italy'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of Relative COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed Cases ', 'value': 'total_cases'},
        {'label': 'Timeline Confirmed Cases Filtered', 'value': 'total_cases_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'total_cases_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'total_cases_filtered_DR'},
    ],
    value='total_cases',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Relative Cases of COVID-19 infected people (source ourworldindata, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['location']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['location','total_cases','total_cases_filtered','total_cases_DR','total_cases_filtered_DR','date']].groupby(['location','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['location','total_cases','total_cases_filtered','total_cases_DR','total_cases_filtered_DR','date']].groupby(['location','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)

C:\Users\pandh\Desktop\EDS2
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
